In [1]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from transformers import ViltProcessor, ViltModel
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [2]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePalooza Data/Benchmarking/Beetlepalooza_beetles_image_only.csv"
df = pd.read_csv(file_path, sep="\t")
df = df[['ImageFilePath', 'Genus']]
df.head(2)

,ImageFilePath,Genus
0,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius
1,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "dandelin/vilt-b32-mlm"

model = ViltModel.from_pretrained(model_name).eval().to(device)

processor = ViltProcessor.from_pretrained(model_name)


In [4]:
def extract_features(image_path, taxa) :
    
    image = Image.open(image_path).convert("RGB")
    
    text = f"This is an image of a beetle, of genus : {taxa}"
    
    inputs = processor(images=image, text=text, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    features = outputs.last_hidden_state.mean(dim=1)
    
    return features.cpu().numpy().flatten()
    

In [5]:
X = np.vstack([extract_features(img, taxa) for img, taxa in tqdm(zip(df["ImageFilePath"], df["Genus"])) ])

le = LabelEncoder()

y = le.fit_transform(df["Genus"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


11399it [26:02,  7.30it/s]


In [6]:
seed = 99

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "SVMPolynomial": SVC(kernel="poly", degree=4),
    "SVMRadialBasis": SVC(kernel="rbf", degree=4),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=seed),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(256, 128), alpha=0.001, max_iter=250, random_state=seed)
}

predictions = {}

metrics = {}

for name, model in models.items():
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


NaiveBayes                | Acc: 93.20% | Prec: 94.19% | Rec: 93.20% | F1: 93.49% | Bal Acc: 84.95% | MCC: 0.9237


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


LogisticRegression        | Acc: 99.78% | Prec: 99.75% | Rec: 99.78% | F1: 99.76% | Bal Acc: 95.87% | MCC: 0.9975


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


...

/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


NearestNeighbor           | Acc: 98.11% | Prec: 98.18% | Rec: 98.11% | F1: 98.01% | Bal Acc: 85.40% | MCC: 0.9788


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


RandomForest              | Acc: 92.81% | Prec: 92.22% | Rec: 92.81% | F1: 91.60% | Bal Acc: 57.55% | MCC: 0.9188
MLP_Baseline              | Acc: 99.82% | Prec: 99.80% | Rec: 99.82% | F1: 99.81% | Bal Acc: 96.12% | MCC: 0.9980


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,Genus,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_SVMPolynomial,Pred_SVMRadialBasis,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
1766,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Synuchus,Synuchus,Synuchus,Synuchus,Synuchus,Synuchus,Synuchus,Synuchus,Synuchus
2629,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Agonum,Agonum,Agonum,Agonum,Calathus,Agonum,Agonum,Synuchus,Agonum


In [8]:
metrics_df

,Model,Accuracy,Precision,Recall,F1-Score,Balanced Acc,MCC
NaiveBayes,NaiveBayes,0.932018,0.941915,0.932018,0.934884,0.849544,0.923741
LogisticRegression,LogisticRegression,0.997807,0.997517,0.997807,0.997618,0.958706,0.997528
SVMLinear,SVMLinear,0.998684,0.998719,0.998684,0.998692,0.965101,0.998517
SVMPolynomial,SVMPolynomial,0.896053,0.916827,0.896053,0.888502,0.661103,0.884777
SVMRadialBasis,SVMRadialBasis,0.992105,0.990916,0.992105,0.991182,0.880127,0.991103
NearestNeighbor,NearestNeighbor,0.98114,0.981829,0.98114,0.980134,0.853997,0.97882
RandomForest,RandomForest,0.92807,0.922212,0.92807,0.915965,0.575517,0.918835
MLP_Baseline,MLP_Baseline,0.998246,0.997962,0.998246,0.998058,0.961169,0.998023


In [9]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/28.ViLT-linear-probing-genus.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/28.ViLT-linear-probing-genus-metrics.csv", index=False)